In [105]:
# import the necessary packages
from imutils import paths 
import argparse
import imutils
import json 
import cv2
import os
import pandas as pd
import numpy as np

# tools for parallelism
from multiprocessing import Pool
from functools import partial

# separate function to compute mask for one landmark
from utils.utils import one_key_mask

In [106]:
# original image size and intended image size
H = 512
W = 640

h = 256
w = 256

# number of facial landmarks
KEYPOINTS = 54

In [115]:
def generate_masks(labelsFolder,bb="no"):
    # bb:0 or 1 , 0 means without bounding_box, 1 means with bounding box
    # folder where generated masks will be stored
    if bb=="yes":
        masksFolder = labelsFolder.replace('/labels', '/bb_masks')
    else:
        masksFolder=labelsFolder.replace('/labels','/masks')
    if not os.path.exists(masksFolder):
        os.makedirs(masksFolder)

    # paths to labels folder
    txtPaths = list(paths.list_files(labelsFolder, validExts="txt"))
    txtPaths = sorted(txtPaths)
    
    #Bounding box json file
    bounding_box=open('dataset/bounding_box.json')
    bb = json.load(bounding_box) 
    bb_key_map = {}
    for k,v in bb.items():
        bb_key_map[k.split('.')[0]] = k
    
    # loop over the text files
    for ind, txtPath in enumerate(txtPaths, 1):
        print("[INFO] Processing {} file ({}/{})".format(txtPath.split("/")[-1], ind, len(txtPaths)))
        # opening the text file 
        f = open(txtPath,) 
        f = f.readlines()
        image_name = txtPath.split("\\")[-1].split(".")[0]
        bb_key = bb_key_map[image_name]
        
        # returns the json object as a dictionary 
        #data = json.load(f) 

        # extracting filename
        filename = os.path.split(txtPath)[-1] #split("/")[-1]
        filename = filename.split(".")[0]
        
        if bb == "yes":
            xs = bb[bb_key]['regions'][0]['shape_attributes']['x']
            ys = bb[bb_key]['regions'][0]['shape_attributes']['y']
            xe = xs+bb[bb_key]['regions'][0]['shape_attributes']['width']
            ye = ys+bb[bb_key]['regions'][0]['shape_attributes']['height']
        else:
            xs = 0
            ys = 0
            xe = W
            ye = H
        # iterating through the shapes
        landmarks = []
        for point in f:
            point=point.split(' ')
            x=point[1]
            y=point[2]
            (x,y)=(W*float(x),H*float(y))
            (crop_x, crop_y) = (x-xs, y-ys)
            (crop_x, crop_y) = ((crop_x / (xe-xs)) * w, (crop_y / (ye-ys)) * h)
            landmarks.append([crop_y, crop_x])
        landmarks = np.array(landmarks)

        # array to contain the masks for each point
        B = np.zeros((KEYPOINTS, h, w))
        # mask generation is computed on CPU
        # hence, the landmark computation is manually divided to 6 processors
        # separate landmarks to 6 subsets for parallel computation
        keys1 = (list(range(0, 9)))
        keys2 = (list(range(9, 18)))
        keys3 = (list(range(18, 27)))
        keys4 = (list(range(27, 36)))
        keys5 = (list(range(36, 45)))
        keys6 = (list(range(45, 54)))

        with Pool(processes=6) as pool:
            b1 = pool.map(partial(one_key_mask, landmarks = landmarks, m=h, n=w), keys1)
            b2 = pool.map(partial(one_key_mask, landmarks = landmarks, m=h, n=w), keys2)
            b3 = pool.map(partial(one_key_mask, landmarks = landmarks, m=h, n=w), keys3)
            b4 = pool.map(partial(one_key_mask, landmarks = landmarks, m=h, n=w), keys4)
            b5 = pool.map(partial(one_key_mask, landmarks = landmarks, m=h, n=w), keys5)
            b6 = pool.map(partial(one_key_mask, landmarks = landmarks, m=h, n=w), keys6)

            B[0:9, :, :] = np.array(b1)
            B[9:18, :, :] = np.array(b2)
            B[18:27, :, :] = np.array(b3)
            B[27:36, :, :] = np.array(b4)
            B[36:45, :, :] = np.array(b5)
            B[45:54, :, :] = np.array(b6)

        # combining to one mask
        mask = np.sum(B, axis=0)
        mask = np.clip(mask, 0, 1)
        mask = mask * 255.0
        mask = mask.astype('uint8')

        # saving the mask
        maskPath = os.path.join(masksFolder, "{}.png".format(filename))
        cv2.imwrite(maskPath, mask)

In [116]:
generate_masks('dataset/labels',"no")

[INFO] Processing labels\1_11178.txt file (1/100)
[INFO] Processing labels\1_11732.txt file (2/100)
[INFO] Processing labels\1_12043.txt file (3/100)
[INFO] Processing labels\1_12260.txt file (4/100)
[INFO] Processing labels\1_12600.txt file (5/100)
[INFO] Processing labels\1_12929.txt file (6/100)
[INFO] Processing labels\1_13014.txt file (7/100)
[INFO] Processing labels\1_14762.txt file (8/100)
[INFO] Processing labels\1_16230.txt file (9/100)
[INFO] Processing labels\1_16323.txt file (10/100)
[INFO] Processing labels\1_17866.txt file (11/100)
[INFO] Processing labels\1_17981.txt file (12/100)
[INFO] Processing labels\1_19078.txt file (13/100)
[INFO] Processing labels\1_19787.txt file (14/100)
[INFO] Processing labels\1_19801.txt file (15/100)
[INFO] Processing labels\1_19961.txt file (16/100)
[INFO] Processing labels\1_20066.txt file (17/100)
[INFO] Processing labels\1_20474.txt file (18/100)
[INFO] Processing labels\1_20498.txt file (19/100)
[INFO] Processing labels\1_20623.txt fil